# MMSB textbook figures

In [ ]:
using DifferentialEquations
using LabelledArrays
using UnPack
using Plots
import Plots.PlotMeasures.mm as pltmmm

Plots.default(linewidth=2)

In [ ]:
# Convenience functions
hil(x, k) = x / (x + k)
hil(x, k, n) = hil(x^n, k^n)
exprel(x) = x / expm1(x)

## Chapter 1

### Fig 1.07 Collins toggle switch

For Figures 1.7, 7.13, 7.14, 7.15

In [ ]:
function collins!(D, u, p, t)
    @unpack a1, a2, β, γ, i1, i2 = p
    @unpack s1, s2 = u
    D.s1 = a1 * hil(1 + i2, s2, β) - s1
    D.s2 = a2 * hil(1 + i1, s1, γ) - s2
    nothing
end

on_10!(integrator) = integrator.p.i2 = 10.
on_20!(integrator) = integrator.p.i2 = 0.
on_30!(integrator) = integrator.p.i1 = 10.
on_40!(integrator) = integrator.p.i1 = 0.

events = CallbackSet(
    PresetTimeCallback(10., on_10!), 
    PresetTimeCallback(20., on_20!),
    PresetTimeCallback(30., on_30!),
    PresetTimeCallback(40., on_40!),
)

In [ ]:
ps = LVector(a1=3.0, a2=2.5, β=4.0, γ=4.0, i1=0.0, i2=0.0)
u0 = LVector(s1=0.075, s2=2.5)
tend = 50.0

prob = ODEProblem(collins!, u0, tend, ps)
sol = solve(prob, callback=events)

plot(sol, legend=:right, xlabel = "Time", ylabel="Concentration", title="Figure 1.7 Collins toggle switch")

### Fig 1.09 Hodgkin-Huxley model

In [ ]:
# Stimulation current
_istim(t) = ifelse(20 <= t <= 21, -6.6, 0.0) + ifelse(60 <= t <= 61, -6.9, 0.0)

# Neuron model
function hh!(D, u, p, t)
    @unpack G_N_BAR, E_N, G_K_BAR, E_K, G_LEAK, E_LEAK, C_M = p
    @unpack v, m, h, n = u
    mα = exprel(-0.10 * (v + 35))
    mβ  = 4.0 * exp(-(v + 60) / 18.0)
    hα  = 0.07 * exp(- ( v + 60) / 20)
    hβ  = 1 / (exp(-(v+30)/10) + 1)
    nα  = 0.1 * exprel(-0.1 * (v+50))
    nβ  = 0.125 * exp( -(v+60) / 80)
    iNa = G_N_BAR * (v - E_N) * (m^3) * h
    iK  = G_K_BAR * (v - E_K) * (n^4)
    iLeak = G_LEAK * (v - E_LEAK)
    iStim = _istim(t)
    D.v = -(iNa + iK + iLeak + iStim) / C_M
    D.m = -(mα + mβ) * m + mα
    D.h = -(hα + hβ) * h + hα
    D.n = -(nα + nβ) * n + nα
    nothing
end

In [ ]:
ps = (
    E_N = 55.0,       # Reversal potential of Na (mV)
    E_K = -72.0,      # Reversal potential of K (mV)
    E_LEAK = -49.0,   # Reversal potential of leaky channels (mV)
    G_N_BAR = 120.0,  # Max. Na channel conductance (mS/cm^2)
    G_K_BAR = 36.0,   # Max. K channel conductance (mS/cm^2)
    G_LEAK = 0.30,    # Max. leak channel conductance (mS/cm^2)
    C_M = 1.0        # membrane capacitance (uF/cm^2))
)
u0 = LVector(v=-59.8977, m=0.0536, h=0.5925, n=0.3192)
tend = 100.0

prob = ODEProblem(hh!, u0, tend, ps)
sol = solve(prob, tstops=[20., 60.])

In [ ]:
p1 = plot(
    sol, idxs=[:v],
    ylabel="Membrane potential (mV)", xlabel="",
    legend=false, title="Figure 1.9 Hodgkin-Huxley model")
p2 = plot(sol, idxs = [:m, :h, :n], xlabel="")
p3 = plot(_istim, sol.t, xlabel="Time (ms)", ylabel="Current",
        labels="Stimulation current")
plot(p1, p2, p3, layout=(3, 1), size=(600, 900), leftmargin=5pltmmm)

## Chapter 2

### Fig 2.04 Exponential decay

In [ ]:
plot(title= "Figure 2.4 Exponential decay")
for k in 1:3
    plot!(t -> 3 * exp(-k*t), 0., 5., label = "exp(-$(k)t)")
end

plot!(xlim = (0, 5), ylim=(0, 3.2), xlabel="Time", ylabel="Concentration")

### Fig 2.09 Metabolic network simulation

Numerical simulation of a metabolic network using `Catalyst.jl`

In [ ]:
using DifferentialEquations
using Catalyst
using ModelingToolkit
using Plots
Plots.default(linewidth=2)

In [ ]:
# Model building
rn = @reaction_network begin
    3., 0 --> A
    2., A --> B
    2.5, A + B --> C + D
    3., C --> 0
    4., D --> 0
end

Converting a reaction network to a system of ODEs.

In [ ]:
osys = convert(ODESystem, rn)
for eq in osys.eqs
    println(eq)
end

In [ ]:
u0 = [:A=>0., :B=>0., :C=>0., :D=>0.]
tend = 10.
sol = solve(ODEProblem(rn, u0, tend))

plot(sol, legend=:bottomright, title="Figure 2.09 Metabolic network", 
    xlims=(0., 4.), ylims=(0., 1.), 
    xlabel="Time (sec)", ylabel="Concentration (mM)")

### Figure 2.11

Model reduction of ODE metabolic networks

In [ ]:
using DifferentialEquations
using Catalyst
using ModelingToolkit
using Plots
Plots.default(linewidth=2)

In [ ]:
rn211 = @reaction_network begin
    k0, 0 --> A
    (k1, km1), A <--> B
    k2, B --> 0
end

In [ ]:
@unpack k0, k1, km1, k2, A, B = rn211
ps1 = [k0=>0., k1=>9., km1=>12., k2=>2.]
u0 = [A=>0., B=>10.]
tend = 3.0
sol211 = solve(ODEProblem(rn211, u0, tend, ps1))

In [ ]:
plot(
    sol211, 
    xlabel="Time (AU)", 
    ylabel="Concentration (AU)", 
    title="Fig. 2.11 (Full model)"
)

### Figure 2.12 : Rapid equilibrium assumption

In [ ]:
using DifferentialEquations
using ModelingToolkit

In [ ]:
function make_212(;name)
    @parameters k0 k1 km1 k2
    @variables t
    @variables A(t) B(t) C(t)
    D = Differential(t)
    eqs = [
        C ~ A + B
        B ~ C * k1 / (km1 + k1)
        D(C) ~ k0 - k2 * B
    ]
    sys = ODESystem(eqs; name)
    structural_simplify(sys)
end

In [ ]:
@named model212 = make_212()

In [ ]:
states(model212)

In [ ]:
observed(model212)

In [ ]:
parameters(model212)

In [ ]:
independent_variables(model212)

In [ ]:
@unpack k0, k1, km1, k2, C, A, B = model212
ps1 = [k0=>0., k1=>9., km1=>12., k2=>2.]
u0 = [C=>10.]
tend = 3.0
prob = ODEProblem(model212, u0, tend, ps1)
sol212 = solve(prob)

In [ ]:
plot(sol211, line=(:dash, 1), label=["A (full solution)" "B (full solution)"])
plot!(sol212, idxs=[A, B], label=["A (rapid equilibrium)" "B (rapid equilibrium)"])
plot!(
    title="Fig. 2.12 (Rapid equilibrium model)", 
    xlabel="Time (AU)", 
    ylabel="Concentration (AU)"
)

### Figure 2.13: Rapid equilibrium (take 2)

When another set of parameters is not suitable for rapid equilibrium assumption.

In [ ]:
ps2 = [k0=>9., k1=>20., km1=>12., k2=>2.]
u0 = [A=>8., B=>4.]
tend = 3.0

In [ ]:
sol213full = solve(ODEProblem(rn211, u0, tend, ps2))
sol213re = solve(ODEProblem(model212, [C => sum(last.(u0))], tend, ps2))

In [ ]:
plot(sol213full, line=(:dash, 1), label=["A (full solution)" "B (full solution)"])
plot!(sol213re, idxs=[A, B], label=["A (rapid equilibrium)" "B (rapid equilibrium)"])
plot!(
    title="Fig. 2.13 (Rapid equilibrium model)", 
    xlabel="Time (AU)", 
    ylabel="Concentration (AU)"
)

### Figure 2.14 : QSSA

Quasi-steady state assumption on species A

In [ ]:
function make_214(;name)
    @parameters k0 k1 km1 k2
    @variables t
    @variables A(t) B(t)
    D = Differential(t)
    eqs = [
        A ~ (k0 + km1 * B)/k1
        D(B) ~ k1 * A - (km1 + k2) * B
    ]
    sys = ODESystem(eqs; name)
    structural_simplify(sys)
end

In [ ]:
@named model214 = make_214()

In [ ]:
sol214 = solve(ODEProblem(model214, [B => (k1 * sum(last.(u0)) - k0) / (k1 + km1)], tend, ps2))

In [ ]:
plot(sol213full, line=(:dash))
plot!(sol214, idxs=[A, B], label=["A (QSSA)" "B (QSSA)"])
plot!(   
    xlabel="Time (AU)",
    ylabel="Concentration (AU)",
    title="Figure 2.14: Ref vs QSSA",
    xlims=(0.0, tend)
)

### Problem 2.4.6

In [ ]:
using DifferentialEquations
using Plots
Plots.default(linewidth=2)

# Using pipe operator |>
ODEProblem((u, p, t) -> p * (1. - u), 0., 10., 1.) |> solve |> plot

## Chapter 3

### Figure 3.03 Michaelis-Menten kinetics

In [ ]:
using DifferentialEquations
using Catalyst
using Plots
Plots.default(linewidth=2)

In [ ]:
rn303 = @reaction_network begin
    (k1, km1), S + E <--> ES
    k2, ES --> E + P 
end

In [ ]:
u0 = [:S=>5., :ES=>0., :E=>1., :P=>0.]
ps = [:k1 => 30., :km1 => 1., :k2 => 10.]
tend = 1.0

In [ ]:
prob = ODEProblem(rn303, u0, tend, ps)
sol = solve(prob)

In [ ]:
plot(sol, xlabel="Time (AU)", ylabel="Concentration (AU)", legend=:right)

In [ ]:
rn303mm = @reaction_network begin
    mm(S, k2 * ET, (km1 + k2) / k1), S ⇒ P # \Rightarrow
end

In [ ]:
u0 = [:S=>5., :P=>0.]
ps = [:k1 => 30., :km1 => 1., :k2 => 10., :ET=>1.]
tend = 1.0
probmm = ODEProblem(rn303mm, u0, tend, ps)
solmm = solve(probmm)

In [ ]:
@unpack S, P = rn303
plot(sol, idxs=[S, P], line=(:dash), label=["S (full)" "P (full)"])
plot!(solmm, idxs=[S, P], label=["S (MM)" "P (MM)"])
plot!(
    title="Fig. 3.03", 
    xlabel="Time (AU)", 
    ylabel="Concentration (AU)", 
    xlims=(0., tend), 
    ylims=(0., 5.), legend=:right)

### Fig 3.13 GMA and Michaelis-Menten rate laws

In [ ]:
using Plots
Plots.default(linewidth=2)

plot(
    [t -> 2t / (1+t), t -> t^0.4], 0., 4., 
    label = ["MM" "GMA"], title = "Fig 3.13",
    xlabel= "Substrate concentration (AU)", 
    ylabel="Reaction rate (AU)"
)

## Chapter 4

### Figure 4.1, 4.2, and 4.3

Steady states and phase plots in an assymetric network.

In [ ]:
using DifferentialEquations
using Catalyst
using ComponentArrays
using Plots
using LinearAlgebra
Plots.default(linewidth=2)

In [ ]:
rn41 = @reaction_network begin
    (hillr(B, k1, 1, n), k3), 0 <--> A
    k5, A --> B
    (k2, k4), B <--> 0
end

In [ ]:
@unpack k1, k2, k3, k4, k5, n, A, B = rn41
ps1 = [k1=>20., k2=>5., k3=>5., k4=>5., k5=>2., n=>4.]

u0s = ([A=>0., B=>0.])

In [ ]:
function model41(u, p, t)
    a, b = u
    @unpack k1, k2, k3, k4, k5, n = p
    da = k1 * hil(1, b, n) - (k3 + k5) * a
    db = k2 + k5 * a - k4 * b
    return (da, db)
end

function model41!(D, u, p, t)
    D.a, D.b = model41(u, p, t)
    return nothing
end

In [ ]:
ps1 = ComponentArray(k1=20., k2=5., k3=5., k4=5., k5=2., n=4.)
u0s = [
    ComponentArray(a=0.0, b=0.0),
    ComponentArray(a=0.5, b=0.6),
    ComponentArray(a=0.17, b=1.1),
    ComponentArray(a=0.25, b=1.9),
    ComponentArray(a=1.85, b=1.7),
]

tend = 1.5

In [ ]:
sols = map(u0s) do u0
    prob = ODEProblem(model41!, u0, tend, ps1)
    sol = solve(prob)
end

plot(sols[1], xlabel="Time", ylabel="Concentration", title="Fig. 4.2 A (Time series)", labels=["[A]" "[B]"])

In [ ]:
plot(sols[1], idxs=(:a, :b), xlabel="[A]", ylabel="[B]", aspect_ratio=:equal,
     title="Fig. 4.2 B (Phase plot)", ylims=(0.0, 2.0), xlims=(0.0, 2.0), 
     legend=nothing)

In [ ]:
p43a = plot(title="Fig. 4.3A (Multiple time series)")
	
for sol in sols
    plot!(p43a, sol, linealpha=0.5, legend=nothing)
end

plot!(p43a, xlabel="Time", ylabel="Concentration")

In [ ]:
p43b = plot(title="Fig. 4.3 B (Phase plot)")

for sol in sols
    plot!(p43b, sol, idxs=(:a, :b), legend=nothing)
end

plot!(p43b, xlabel="[A]", ylabel="[B]", xlims=(0., 2.), ylims=(0., 2.), aspect_ratio=:equal)

### Figure 4.4 and 4.5 

Vector fields in phase plots.

In [ ]:
# Mesh points
r = range(0.0, 2.0, 41)
xy = [(x, y) for y in r, x in r]
xx = first.(xy)
yy = last.(xy)

dadb = map(xx, yy) do x, y
    model41((x, y), ps1, 0.0)
end

da = first.(dadb)
db = last.(dadb)

In [ ]:
scale = 20 .* (hypot.(da, db).^0.5)
da2 = da ./ scale
db2 = db ./ scale

quiver(
    xx[1:4:end], yy[1:4:end], quiver=(da2[1:4:end], db2[1:4:end]),
    line=(:lightgrey), aspect_ratio=:equal, arrow=(:closed),
    xlims=(r[1], r[end]), ylims=(r[1], r[end]))

contour!(r, r, da, levels=[0], cbar=false, line=(:black))
plot!(identity, 0., 0., line=(:black), label="A nullcline")
contour!(r, r, db, levels=[0], cbar=false, line=(:black, :dash))
plot!(identity, 0., 0., line=(:black, :dash), label="B nullcline")
plot!(title="Nullclines", legend=:bottomleft)

In [ ]:
p44a = plot(title="Fig. 4.4 A (Phase plot with vector field)")

for sol in sols
    plot!(p44a, sol, idxs=(:a, :b), linealpha=0.7, legend = nothing)
end

quiver!(p44a,
    xx[1:4:end], yy[1:4:end], quiver=(da2[1:4:end], db2[1:4:end]),
    line=(:lightgrey), aspect_ratio=:equal, arrow=(:closed),
    xlims=(r[1], r[end]), ylims=(r[1], r[end]),
    xlabel="[A]", ylabel="[B]", size=(600, 600)
)

In [ ]:
# Figure 4.5A
p45a = plot(title="Fig. 4.5 A (Phase plot with nullclines)")

# Phase plots
for sol in sols
    plot!(p45a, sol, idxs=(:a, :b), linealpha=0.7, lab=nothing)
end

# nullclines
contour!(p45a, r, r, da, levels=[0], cbar=false, line=(:black))
plot!(p45a, identity, 0., 0., line=(:black), label="A nullcline")
contour!(p45a, r, r, db, levels=[0], cbar=false, line=(:black, :dash))
plot!(p45a, identity, 0., 0., line=(:black, :dash), label="B nullcline")
plot!(p45a, xlim=(0., 2.), ylim=(0., 2.), legend=:bottomright, size=(600, 600), xlabel="[A]", ylabel="[B]", aspect_ratio=:equal)

In [ ]:
quiver(
    xx[1:4:end], yy[1:4:end], quiver=(da2[1:4:end], db2[1:4:end]),
    line=(:lightgrey), arrow=(:closed),
    xlims=(r[1], r[end]), ylims=(r[1], r[end]))

contour!(r, r, da, levels=[0], cbar=false, line=(:black))
plot!(identity, 0., 0., line=(:black), label="A nullcline")
contour!(r, r, db, levels=[0], cbar=false, line=(:black, :dash))
plot!(identity, 0., 0., line=(:black, :dash), label="B nullcline")
plot!(title="Fig. 4.5 B (Vector field with nullclines)", xlabel="[A]", ylabel="[B]", aspect_ratio=:equal, xlim=(0., 2.), ylim=(0., 2.), legend=:bottomright, size=(600, 600))

### Figure 4.7, 4.8, 4.9, and 4.19A

Symmetric (bistable) biological networks.

In [ ]:
using DifferentialEquations
using ComponentArrays
using UnPack
using Plots
using LinearAlgebra
Plots.default(linewidth=2, fmt=:png)

In [ ]:
function model47!(D, u, p, t)
    @unpack k1, k2, k3, k4, n1, n2 = p
    @unpack a, b = u
    D.a = k1 * hil(1, b, n1) - k3 * a
    D.b = k2 * hil(1, a, n2) - k3 * b
end

In [ ]:
ps1 = ComponentArray(k1=20., k2=20., k3=5., k4=5., n1=4., n2=1.)
tend = 4.0

sol1 = solve(ODEProblem(model47!, ComponentArray(a=3.,b=1.), tend, ps1))
sol2 = solve(ODEProblem(model47!, ComponentArray(a=1.,b=3.), tend, ps1))

p47a1 = plot(sol1, xlabel="Time", ylabel="Concentration", legend=:right, title= "Fig 4.7A (1)")
p47a2 = plot(sol2, xlabel="Time", ylabel="Concentration", legend=:right, title= "Fig 4.7A (2)")
fig47a = plot(p47a1, p47a2, layout=(2, 1), size=(600, 600))

In [ ]:
r = range(0., 5., 21)
xx = [x for y in r, x in r]
yy = [y for y in r, x in r];

In [ ]:
nca47(b, p) = p.k1 / p.k3 * hil(1, b, p.n1)
ncb47(a, p) = p.k2 / p.k4 * hil(1, a, p.n2)

function ∂F47(a, b, p; scale=20)
    u = ComponentArray(a=a, b=b)
    D = copy(u)
    model47!(D, u, p, nothing)
	return D ./ (norm(D)^0.5 * scale)
end

In [ ]:
p47b = quiver(xx, yy, quiver=(x, y)-> ∂F47(x, y, ps1; scale=20), line=(:lightgrey), arrow=(:closed), aspect_ratio=:equal, size =(600, 600))
plot!(p47b, b -> nca47(b, ps1), identity, 0., 5., label="Nullcline A", line=(:dash, :red))
plot!(p47b, identity, a -> ncb47(a, ps1), 0., 5., label="Nullcline B", line=(:dash, :blue))
plot!(p47b, title="Fig 4.7 B", xlim=(0., 5.), ylim=(0., 5.), aspect_ratio=:equal, size =(600, 600), xlabel="[A]", ylabel="[B]")

In [ ]:
ps2 = ComponentArray(k1=20., k2=20., k3=5., k4=5., n1=4., n2=4.)

tend = 4.0
sol1 = solve(ODEProblem(model47!, ComponentArray(a=3.,b=1.), tend, ps2))
sol2 = solve(ODEProblem(model47!, ComponentArray(a=1.,b=3.), tend, ps2))

pl48a1 = plot(sol1, xlabel="Time", ylabel="Concentration", legend=:right, title= "Fig 4.8A (1)")
pl48a2 = plot(sol2, xlabel="Time", ylabel="Concentration", legend=:right, title= "Fig 4.8A (2)")
fig48a = plot(pl48a1, pl48a2, layout=(2, 1), size=(600, 600))

In [ ]:
p48b = quiver(xx, yy, quiver=(x, y)-> ∂F47(x, y, ps2; scale=20), line=(:lightgrey), arrow=(:closed), aspect_ratio=:equal, size =(600, 600))
plot!(p48b, b -> nca47(b, ps2), identity, 0., 5., label="Nullcline A", line=(:dash, :red))
plot!(p48b, identity, a -> ncb47(a, ps2), 0., 5., label="Nullcline B", line=(:dash, :blue))
plot!(p48b, title="Fig 4.8 B", xlim=(0., 5.), ylim=(0., 5.), aspect_ratio=:equal, size =(600, 600), xlabel="[S1]", ylabel="[S2]")

In [ ]:
r2 = range(1.0, 1.5, 11)
xx2 = [x for y in r2, x in r2]
yy2 = [y for y in r2, x in r2]

p48b = quiver(xx2, yy2, quiver=(x, y)-> ∂F47(x, y, ps2; scale=60), line=(:lightgrey), arrow=(:closed), aspect_ratio=:equal, size =(600, 600))
plot!(p48b, b -> nca47(b, ps2), identity, r2[1], r2[end], label="Nullcline A", line=(:dash, :red))
plot!(p48b, identity, a -> ncb47(a, ps2), r2[1], r2[end], label="Nullcline B", line=(:dash, :blue))
plot!(p48b, title="Fig 4.8 B (close up)", xlim=(r2[1], r2[end]), ylim=(r2[1], r2[end]), aspect_ratio=:equal, size =(600, 600), xlabel="[S1]", ylabel="[S2]")

In [ ]:
pls = map((8.0, 16.0, 20.0, 35.0)) do k1
    ps = ComponentArray(k1=k1, k2=20., k3=5., k4=5., n1=4., n2=4.)
    pl = plot(b -> nca47(b, ps), identity, 0., 7., label="Nullcline A")
    plot!(pl, identity, a -> ncb47(a, ps), 0., 7., label="Nullcline B")
    plot!(pl, title = "K1 = $k1", xlim=(0., 7.), ylim=(0., 7.), aspect_ratio = :equal, xlabel="[A]", ylabel="[B]")
    pl
end

plot(pls..., size = (600, 600))

### Figure 4.11

Surface plots reference: [surface plots @ PlotsGallery.jl](https://goropikari.github.io/PlotsGallery.jl/src/surface.html)

In [ ]:
using Plots
Plots.default(linewidth=2, fmt=:png)

z1(x, y) = x^2 + 0.5y^2
z2(x, y) = (.2x^2-1)^2 + y^2
x1 = y1 = range(-1.0, 1.0, 41)
x2 = range(-2.75, 2.75, 41)
y2 = range(-0.75, 0.75, 41)
p1 = surface(x1, y1, z1, title="Single-well potential")
p2 = contourf(x1, y1, z1)
p3 = surface(x2, y2, z2, title="Double-well potential")
p4 = contourf(x2, y2, z2)

plot(p1, p2, p3, p4, size=(800, 600))

### Figure 4.15, 4.16, and 4.17

Oscillatory networks.

In [ ]:
using DifferentialEquations
using ComponentArrays
using LinearAlgebra
using Plots
Plots.default(linewidth=2, fmt=:png)

In [ ]:
function model415!(D, u, p, t)
    @unpack k0, k1, k2, n = p
    @unpack a, b = u
    vab = k1 * a * (1 + b^n)
    D.a = k0 - vab
    D.b = vab - k2 * b
    return nothing
end

In [ ]:
function figure0415(; 
    ps = ComponentArray(k0 = 8., k1 = 1., k2 = 5., n = 2.),
	r = range(0., 4., 21),
	tend = 8.,
	figtitle="Fig 4.15",
    model = model415!,
    u0s = (
        ComponentArray(a=1.5, b=1.0),
        ComponentArray(a=0.0, b=1.0), 
        ComponentArray(a=0.0, b=3.0),
        ComponentArray(a=2.0, b=0.0),
    )
)
	sols = map(u0s) do u0
		solve(ODEProblem(model, u0, tend, ps))
	end

	p1 = plot(sols[1], xlabel="Time", ylabel="Concentration", title ="$figtitle (A)", xlims=(0., 8.))

    function ∂F(x, y; scale=20)
        u = ComponentArray(a=x, b=y)
        D = copy(u)
        model415!(D, u, ps, nothing)
        return D ./ (norm(D)^0.5 * scale)
    end

	nc_a(b) = ps.k0 / ps.k1 * hil(1, b, ps.n)
    nc_b(b) = (ps.k2 * b) / (ps.k1 * (1 + b ^ ps.n))
	
	xx = [x for y in r, x in r]
	yy = [y for y in r, x in r]

	p2 = plot(title = "$figtitle (B)")
	for sol in sols
		plot!(p2, sol, idxs=(:a, :b), label=nothing)
	end
	
	rMin, rMax = r[begin], r[end]
	
	plot!(p2, nc_a, identity, rMin, rMax, label="Nullcline A", line=(:dash, :red))
	plot!(p2, nc_b, identity, rMin, rMax, label="Nullcline B", line=(:dash, :blue))
    quiver!(p2, xx, yy, quiver=∂F, line=(:lightgrey), xlims=(rMin, rMax), ylims=(rMin, rMax), 
        aspect_ratio=:equal, size=(600, 600), arrow=(:closed), xlabel="[A]", ylabel="[B]")
	return (p1, p2)
end

In [ ]:
fig415a, fig415b = figure0415()

In [ ]:
fig415a

In [ ]:
fig415b

In [ ]:
fig416a, fig416b = figure0415(ps = ComponentArray(k0 = 8., k1 = 1., k2 = 5., n = 2.5) , tend = 1000.0, figtitle="Fig 4.16")

In [ ]:
fig416a

In [ ]:
fig416b

In [ ]:
_, fig417 = figure0415(
    ps = ComponentArray(k0 = 8., k1 = 1., k2 = 5., n = 2.5), 
    tend = 10., r = range(0., 4., 21), 
    u0s = ( ComponentArray(a=2.0, b=1.5),),
)

In [ ]:
plot!(fig417, xlims=(1.0, 3.0), ylims=(1.0, 3.0), title="Fig 4.17")

### Figure 4.18 Continuation diagram

See also [BifurcationKit.jl](https://github.com/bifurcationkit/BifurcationKit.jl)

In [ ]:
using DifferentialEquations
using ComponentArrays
using LinearAlgebra
using Plots
Plots.default(linewidth=2, fmt=:png)

In [ ]:
function model418!(D, u, p, t)
    @unpack a, b = u
    @unpack k1, k2, k3, k4, k5, n = p
    D.a = k1 * hil(1, b, n) - (k3 + k5) * a
    D.b = k2 + k5 * a - k4 * b
end

In [ ]:
ps = ComponentArray(k1 = 20.0, k2 = 5.0, k3 = 5.0, k4 = 5.0, k5 = 2.0, n = 4)
u0 = ComponentArray(a=0., b=0.)
tend = 1000.

In [ ]:
# Could also use parallel ensemble: https://diffeq.sciml.ai/stable/features/ensemble/

r = range(0., 1000., 51)

aInf = map(r) do k1
    p = copy(ps)
    p[1] = k1
	prob = ODEProblem(model418!, u0, tend, p)
	sol = solve(prob, Rodas5(), save_everystep=false)
	sol.u[end].a
end

plot(r, aInf, title = "Fig 4.18 Continuation diagram", 
     xlabel = "K1" , ylabel= "Steady state [A]", 
     legend=nothing, ylim=(0, 4), xlim=(0, 1000))

### Figure 4.22 Tangent line

In [ ]:
using Plots
Plots.default(linewidth=2, fmt=:png)

plot(t -> 3 / (t-2), 2.2, 8.0, lab="Curve")
plot!(t -> 1.5 - (t - 4) * 0.75, 2.7, 5.3, lab="Tangent line")
plot!(title="Fig 4.22", xlabel="Reaction rate", ylabel="Inhibitor concentration", 
      xlims=(2., 8.), ylims=(0., 4.))

## Runtime information

In [ ]:
versioninfo()

In [ ]:
using Pkg
Pkg.status()